In [ ]:
from sklearn.cluster import KMeans
import numpy as np

In [ ]:
x1_txt = "../data/x1.txt"
x2_txt = "../data/x2.txt"
xeasy_txt = "../data/xeasy.txt"

In [ ]:
x1 = np.loadtxt(x1_txt)
x2 = np.loadtxt(x2_txt)
xeasy = np.loadtxt(xeasy_txt)

# xeasy

In [ ]:
model = Kmeans(n_clusters=2).fit(xeasy)
parameters = model.get_params()

# x1

In [ ]:
model = Kmeans(n_clusters=2).fit(x1)
parameters = model.get_params()

# x2

In [ ]:
model = Kmeans(n_clusters=2).fit(x2)
parameters = model.get_params()